In [ ]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import itertools
from scipy.spatial.distance import squareform
import palsgraph

#### Load distance matrix

In [ ]:
simfilename = 'test.tsv'

X = np.genfromtxt(simfilename, delimiter='\t', encoding='utf8', dtype=None)

labels = []
for label in [x[0] for x in X]:
    if label in labels:
        continue
    labels.append(label)
for label in [x[1] for x in X]:
    if label in labels:
        continue
    labels.append(label)

dismat = squareform([x[2] for x in X])
dismat

#### Construct graph

In [ ]:
threshold = 0.77 # Replace this with your choice of threshold

# Initialize similarity matrix
simmat = dismat.copy().reshape((-1,))

# Preparation: remember the indices of the disconnected entries
unconnected_indices = (simmat > threshold)

# Convert distance to similarity
simmat = np.exp(-np.square(simmat))

# Set disconnected entries to zero
simmat[unconnected_indices] = 0
print("{} out of {} values set to zero".format(len(simmat[simmat == 0]), len(simmat)))

# Restore shape
simmat = simmat.reshape(dismat.shape)

# Fill diagonal with the maximum affinity
np.fill_diagonal(simmat, np.max(simmat))

G = palsgraph.make_graph(simmat, labels=labels)

#### Community detection using Girvan-Newman

In [ ]:
from networkx.algorithms.community.centrality import girvan_newman

comp = girvan_newman(G)

max_shown = 3
shown_count = 1
possibilities = []
for communities in itertools.islice(comp, max_shown):
    print("Possibility", shown_count, ": ", end='')

    pos = palsgraph.getpos(G, communities)
    color_map = palsgraph.gen_colormap(G, communities)

    shown_count += 1

    plt.figure(figsize=(17, 15))
    nx.draw(G, pos=pos, node_color=color_map, edge_color='grey', with_labels=True)
    plt.show()